# Rossman Pharmaceutical Sales Prediction

## Imports

In [43]:
import numpy as np
import pandas as pd
import mlflow
import warnings
import pickle
from time import strftime, gmtime
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib import ticker
from statsmodels.tsa.stattools import adfuller, acf, pacf
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [6]:
# sns.set()
%matplotlib inline
warnings.filterwarnings("ignore")

In [8]:
import sys, os

In [46]:
sys.path.append(os.path.abspath(os.path.join('C:\\Data Science\\project-6-Pharmaceutical\\Rossmann-Sales\\scripts')))
from file_handler import FileHandler

## Preprocessing

We have alreadyseen which features influence sales in the exploratory section. And we have used many of the columns for the 4 machine learning models used as seen in the scripts. But for LSTM only the date and sales columns will be used.

In [10]:
file_handler = FileHandler()
train_sales = file_handler.read_csv("C:\\Data Science\\project-6-Pharmaceutical\\Rossmann-Sales\\data\\train.csv")

In [11]:
data = train_sales.groupby("Date").agg({"Sales": "mean"})
data

,Sales
Date,
2013-01-01,87.284560
2013-01-02,6233.030493
2013-01-03,5693.112108
2013-01-04,5954.218834
2013-01-05,5337.751570
...,...
2015-07-27,9602.952466
2015-07-28,8174.953363
2015-07-29,7623.284305


In [12]:
scaler = StandardScaler()
scaled_array = scaler.fit_transform(data)
data['DataScaled'] = scaled_array


In [13]:
SIZE = len(data.DataScaled)
WINDOW_SIZE = 48
BATCH_SIZE = SIZE - WINDOW_SIZE * 2
EPOCHS = 200

In [14]:
DateTrain = data.index.values[0:BATCH_SIZE]
DateValid = data.index.values[BATCH_SIZE:]
XTrain = data.DataScaled.values[0:BATCH_SIZE].astype('float32')
XValid = data.DataScaled.values[BATCH_SIZE:].astype('float32')

# Obtain shapes for vectors of size (,1) for dates series

DateTrain = np.reshape(DateTrain, (-1, 1))
DateValid = np.reshape(DateValid, (-1, 1))

print("Shape of the training set date series: ", DateTrain.shape)
print("Shape of the validation set date series: ", DateValid.shape)
print()
print("Shape of the training set logarithm of sales series: ", XTrain.shape)
print("Shape of the validation set logarithm of sales series in a stateless LSTM: ", XValid.shape)


Shape of the training set date series:  (846, 1)
Shape of the validation set date series:  (96, 1)

Shape of the training set logarithm of sales series:  (846,)
Shape of the validation set logarithm of sales series in a stateless LSTM:  (96,)


In [15]:
tf.random.set_seed(1234)
# add extra dimension
series = tf.expand_dims(XTrain, axis=-1)
series.shape


TensorShape([846, 1])

In [16]:
# create tensor from each individual element
dataset = tf.data.Dataset.from_tensor_slices(series)
dataset


<_TensorSliceDataset element_spec=TensorSpec(shape=(1,), dtype=tf.float32, name=None)>

In [17]:
# takes a window_size + 1 chunk from the slices
dataset = dataset.window(WINDOW_SIZE + 1, shift=1, drop_remainder=True)


In [18]:
# Example of Window
datasetEx = tf.data.Dataset.from_tensor_slices(tf.range(10))
datasetEx = datasetEx.window(5, shift=1, drop_remainder=True)
for window in datasetEx:
    print([elem.numpy() for elem in window])


[np.int32(0), np.int32(1), np.int32(2), np.int32(3), np.int32(4)]
[np.int32(1), np.int32(2), np.int32(3), np.int32(4), np.int32(5)]
[np.int32(2), np.int32(3), np.int32(4), np.int32(5), np.int32(6)]
[np.int32(3), np.int32(4), np.int32(5), np.int32(6), np.int32(7)]
[np.int32(4), np.int32(5), np.int32(6), np.int32(7), np.int32(8)]
[np.int32(5), np.int32(6), np.int32(7), np.int32(8), np.int32(9)]


In [19]:
dataset = dataset.flat_map(lambda window: window.batch(WINDOW_SIZE + 1))


In [20]:
dataset = dataset.map(lambda window: (window[:-1], window[-1:]))


In [21]:
dataset = dataset.batch(BATCH_SIZE).prefetch(1)


In [22]:
def windowed_dataset(series, window_size=WINDOW_SIZE, batch_size=BATCH_SIZE):
  series = tf.expand_dims(series, axis=-1)
  dataset = tf.data.Dataset.from_tensor_slices(series)
  dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
  dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
  dataset = dataset.map(lambda window: (window[:-1], window[-1:]))
  dataset = dataset.batch(batch_size).prefetch(1)
  return dataset


In [23]:
DatasetTrain = windowed_dataset(XTrain)
DatasetVal = windowed_dataset(XValid)


## Model Training

In [24]:
model = Sequential()
model.add(LSTM(8, input_shape=[None, 1], return_sequences=True))
model.add(LSTM(4, input_shape=[None, 1]))
model.add(Dense(1))
model.compile(loss="huber_loss", optimizer='adam')


In [25]:
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, None, 8)        │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 4)              │           208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │             5 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 533 (2.08 KB)

 Trainable params: 533 (2.08 KB)

 Non-trainable params: 0 (0.00 B)

In [27]:
from tensorflow.keras import losses

model.compile(optimizer='adam', loss=losses.Huber())


In [28]:
mlflow.set_experiment("LSTM")
mlflow.tensorflow.autolog()
history = model.fit(DatasetTrain, epochs=EPOCHS, validation_data=DatasetVal, verbose=1)

2024/10/31 13:03:59 WARNING mlflow.utils.autologging_utils: MLflow tensorflow autologging is known to be compatible with 2.7.4 <= tensorflow <= 2.17.0, but the installed version is 2.18.0. If you encounter errors during autologging, try upgrading / downgrading tensorflow to a compatible version, or try upgrading MLflow.
2024/10/31 13:04:00 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '0e0345307dfe46a29ec11d67f420ae41', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow


Epoch 1/200
      1/Unknown 4s 4s/step - loss: 0.4172

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - loss: 0.4172 - val_loss: 0.4082
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step - loss: 0.4164

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 233ms/step - loss: 0.4164 - val_loss: 0.4071
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step - loss: 0.4157

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step - loss: 0.4157 - val_loss: 0.4061
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step - loss: 0.4151

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 333ms/step - loss: 0.2902 - val_loss: 0.2335
Epoch 197/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step - loss: 0.2887

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 303ms/step - loss: 0.2887 - val_loss: 0.2316
Epoch 198/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step - loss: 0.2873

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 332ms/step - loss: 0.2873 - val_loss: 0.2295
Epoch 199/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step - loss: 0.2858

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 300ms/step - loss: 0.2858 - val_loss: 0.2275
Epoch 200/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 308ms/step - loss: 0.2843

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 429ms/step - loss: 0.2843 - val_loss: 0.2254
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 316ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 316ms/step


In [44]:


# Assuming you have trained your model
# Replace colons with dashes for Windows compatibility
time = strftime("%Y-%m-%d-%H-%M-%S", gmtime())  # Changed from %H:%M:%S to %H-%M-%S

# Save the model architecture and weights
model_config = model.to_json()  # Get model architecture
# Save weights in the correct format
model.save_weights(f'C:\\Data Science\\project-6-Pharmaceutical\\Rossmann-Sales\\models\\LSTM_sales_weights_{time}.weights.h5')

# Save model architecture to a .pkl file
with open(f'C:\\Data Science\\project-6-Pharmaceutical\\Rossmann-Sales\\models\\LSTM_sales_{time}.pkl', 'wb') as f:
    pickle.dump(model_config, f)

In [ ]:
model.save('path_to_model.h5')
model.save_weights('path_to_weights.h5')


NameError: name 'model' is not defined